# Import

In [ ]:
### 데이터 조작 라이브러리
import pandas as pd
import numpy as np

### 시각화 라이브러리
import matplotlib.pyplot as plt

### 머신러닝 라이브러리
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

### 전처리 라이브러리
from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE

import warnings

warnings.filterwarnings(action="ignore")

# Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 디렉토리 이동
% cd /content/drive/MyDrive/Colab Notebooks/data_analysis_2025

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/data_analysis_2025/99. 과제형 프로젝트/★ 02. 머신러닝 과제/open


In [ ]:
train = pd.read_csv("./train.csv")
train.info()   # 분야, 직원 수, 고객수(백만명), 기업가치(백억원) : .isna() (True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4376 non-null   object 
 1   설립연도            4376 non-null   int64  
 2   국가              4376 non-null   object 
 3   분야              3519 non-null   object 
 4   투자단계            4376 non-null   object 
 5   직원 수            4202 non-null   float64
 6   인수여부            4376 non-null   object 
 7   상장여부            4376 non-null   object 
 8   고객수(백만명)        3056 non-null   float64
 9   총 투자금(억원)       4376 non-null   float64
 10  연매출(억원)         4376 non-null   float64
 11  SNS 팔로워 수(백만명)  4376 non-null   float64
 12  기업가치(백억원)       3156 non-null   object 
 13  성공확률            4376 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 478.8+ KB


In [ ]:
train.head(2)

,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
0,TRAIN_0000,2009,CT005,이커머스,Series A,4126.0,No,No,56.0,3365.0,4764.0,4.71,NaN,0.3
1,TRAIN_0001,2023,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,1.00,2500-3500,0.8


In [ ]:
train.국가.value_counts()

,count
국가,
CT001,476
CT003,454
CT010,453
CT005,446
CT008,444
CT002,436
CT004,427
CT007,424
CT009,410


In [ ]:
categorical_cols = ['국가', '분야', '투자단계', '인수여부', '상장여부']
numeric_cols = ['설립연도', '직원 수', '고객수(백만명)', '총 투자금(억원)', '연매출(억원)', 'SNS 팔로워 수(백만명)', '성공확률']

In [ ]:
# for col in categorical_cols:
#     plt.figure(figsize=(10, 5))
#     train[col].value_counts().plot(kind='bar')
#     plt.title(f'{col} 분포')
#     plt.xlabel(col)
#     plt.ylabel('빈도')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()


In [ ]:
train.dropna(inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2528 entries, 1 to 4375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              2528 non-null   object 
 1   설립연도            2528 non-null   int64  
 2   국가              2528 non-null   object 
 3   분야              2528 non-null   object 
 4   투자단계            2528 non-null   object 
 5   직원 수            2528 non-null   float64
 6   인수여부            2528 non-null   object 
 7   상장여부            2528 non-null   object 
 8   고객수(백만명)        2528 non-null   float64
 9   총 투자금(억원)       2528 non-null   float64
 10  연매출(억원)         2528 non-null   float64
 11  SNS 팔로워 수(백만명)  2528 non-null   float64
 12  기업가치(백억원)       2528 non-null   object 
 13  성공확률            2528 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 296.2+ KB


In [ ]:
# for col in train.columns:
#     print(train[col].value_counts())

# Data Preprocessing

In [ ]:
### 결측치 처리

In [ ]:
# (1) 분야 : Unknown
# (2) 직원 수 : Imputer
# (3) 고객수(백만명) : Imputer
# (4) 기업가치(백억원) : 평균값 → int

In [ ]:
# # (1) 분야
# train["분야"].fillna("Unknown", inplace=True)  # 또는 'Unknown', 'Missing'
# train["분야"].value_counts()
# # train["분야"]

,count
분야,
Unknown,857
에너지,396
기술,389
에듀테크,364
게임,359
AI,356
핀테크,344
푸드테크,335
이커머스,332


In [ ]:
# # (2) 직원 수, (3) 고객수(백만명)
# from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=5)
# train[['직원 수', '고객수(백만명)']] = imputer.fit_transform(train[['직원 수', '고객수(백만명)']])
# train["직원 수"].isna().sum()

np.int64(0)

In [ ]:
# (4) 기업가치(백억원)
def convert_valuation(val):
    if pd.isna(val):
        return np.nan
    elif isinstance(val, str) and '-' in val:
        low, high = map(int, val.split('-'))
        return (low + high) / 2
    elif isinstance(val, str) and '이상' in val:
        return float(val.split('이상')[0])

train["기업가치(백억원)"] = train["기업가치(백억원)"].apply(convert_valuation)

# 결측치 KNNImputer
# train[["기업가치(백억원)"]] = imputer.fit_transform(train[["기업가치(백억원)"]])
# train[["기업가치(백억원)"]].isna().sum()

In [ ]:
# print(train["기업가치(백억원)"].isna().sum())
print(train["기업가치(백억원)"].value_counts())

기업가치(백억원)
6000.0    527
3000.0    508
5250.0    502
2000.0    500
4000.0    491
Name: count, dtype: int64


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2528 entries, 1 to 4375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              2528 non-null   object 
 1   설립연도            2528 non-null   int64  
 2   국가              2528 non-null   object 
 3   분야              2528 non-null   object 
 4   투자단계            2528 non-null   object 
 5   직원 수            2528 non-null   float64
 6   인수여부            2528 non-null   object 
 7   상장여부            2528 non-null   object 
 8   고객수(백만명)        2528 non-null   float64
 9   총 투자금(억원)       2528 non-null   float64
 10  연매출(억원)         2528 non-null   float64
 11  SNS 팔로워 수(백만명)  2528 non-null   float64
 12  기업가치(백억원)       2528 non-null   float64
 13  성공확률            2528 non-null   float64
dtypes: float64(7), int64(1), object(6)
memory usage: 296.2+ KB


In [ ]:
### 데이터 통합

In [ ]:
strColumnList = ["국가","분야","투자단계","인수여부","상장여부"]
intColumnList = ["설립연도","직원 수"]
floatColumnList = ["고객수(백만명)","총 투자금(억원)","연매출(억원)","SNS 팔로워 수(백만명)","성공확률","기업가치(백억원)"]
# boolColumnList = ["인수여부", "상장여부"]

In [ ]:
for i in range(0, len(strColumnList)):
    targetColumn = strColumnList[i]
    train[targetColumn] = train[targetColumn].astype(str)

for i in range(0, len(intColumnList)):
    targetColumn = intColumnList[i]
    train[targetColumn] = train[targetColumn].astype(int)

for i in range(0, len(floatColumnList)):
    targetColumn = floatColumnList[i]
    train[targetColumn] = train[targetColumn].astype(float)

In [ ]:
# for col in strColumnList:
#     # if len(train[col].unique()) <= 2:
#     print(f"{col}: {len(train[col].unique())}")

# print(train["투자단계"].unique())
# print(train["기업가치(백억원)"].unique())

국가: 10
분야: 11
투자단계: 5
인수여부: 2
상장여부: 2
기업가치(백억원): 6
['Series A' 'Seed' 'Series C' 'Series B' 'IPO']
['nan' '2500-3500' '3500-4500' '1500-2500' '4500-6000' '6000이상']


In [ ]:
### 특성 숫자컬럼 추가

In [ ]:
train = train.drop(columns=["ID"], axis = 1 )

In [ ]:
ynDict = {"Yes":1, "No":0 }
train["인수여부"] = train.인수여부.map(ynDict)
train["상장여부"] = train.상장여부.map(ynDict)
# from sklearn.preprocessing import LabelEncoder

# bool_features = ["인수여부", "상장여부"]
# # categorical_features = [col for col in train.columns if train[col].dtype == 'object']
# print(bool_features)

# encoder = LabelEncoder()
# for feature in bool_features:
#     train[feature] = encoder.fit_transform(train[feature])

train.head()

,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
1,2023,CT006,핀테크,Seed,4167,1,0,80.0,4069.0,279.0,1.0,3000.0,0.8
2,2018,CT007,기술,Series A,3132,1,1,54.0,6453.0,12141.0,4.0,4000.0,0.5
4,2020,CT002,에듀테크,Seed,1969,0,1,94.0,829.0,9810.0,1.0,2000.0,0.1
5,2012,CT008,기술,Series C,3801,0,1,69.0,6505.0,12722.0,3.0,3000.0,0.6
7,2020,CT002,게임,Series A,2617,0,0,80.0,5579.0,5399.0,4.0,4000.0,0.7


In [ ]:
train.국가.drop_duplicates()

,국가
1,CT006
2,CT007
4,CT002
5,CT008
10,CT001
13,CT010
15,CT009
16,CT003
61,CT005
64,CT004


In [ ]:
train.분야.drop_duplicates()

,분야
1,핀테크
2,기술
4,에듀테크
7,게임
10,이커머스
14,헬스케어
16,물류
23,AI
46,에너지
97,푸드테크


In [ ]:
train.투자단계.drop_duplicates()
# seed - series A - series B - series C - IPO

,투자단계
1,Seed
2,Series A
5,Series C
15,IPO
64,Series B


In [ ]:
# 단계별 평균 투자금 (억 원 기준)
investment_map = {
    'Seed': 5,
    'Series A': 50,
    'Series B': 175,
    'Series C': 500,
    'IPO': 2000
}

# 자연 로그 스케일로 변환
train['투자단계'] = train['투자단계'].map(investment_map)
train['투자단계'] = np.log(train['투자단계'])

train.head()

,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
1,2023,CT006,핀테크,1.609438,4167,1,0,80.0,4069.0,279.0,1.0,3000.0,0.8
2,2018,CT007,기술,3.912023,3132,1,1,54.0,6453.0,12141.0,4.0,4000.0,0.5
4,2020,CT002,에듀테크,1.609438,1969,0,1,94.0,829.0,9810.0,1.0,2000.0,0.1
5,2012,CT008,기술,6.214608,3801,0,1,69.0,6505.0,12722.0,3.0,3000.0,0.6
7,2020,CT002,게임,3.912023,2617,0,0,80.0,5579.0,5399.0,4.0,4000.0,0.7


In [ ]:
########
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2528 entries, 1 to 4375
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   설립연도            2528 non-null   int64  
 1   국가              2528 non-null   object 
 2   분야              2528 non-null   object 
 3   투자단계            2528 non-null   float64
 4   직원 수            2528 non-null   int64  
 5   인수여부            2528 non-null   int64  
 6   상장여부            2528 non-null   int64  
 7   고객수(백만명)        2528 non-null   float64
 8   총 투자금(억원)       2528 non-null   float64
 9   연매출(억원)         2528 non-null   float64
 10  SNS 팔로워 수(백만명)  2528 non-null   float64
 11  기업가치(백억원)       2528 non-null   float64
 12  성공확률            2528 non-null   float64
dtypes: float64(7), int64(4), object(2)
memory usage: 276.5+ KB


In [ ]:
# 그룹화할 컬럼 설정
groupKey = ["국가", "분야"]

# 그룹화 및 크기 계산
groupData = train.groupby(by=groupKey).size().reset_index(name = "MAX_KNOB")
# groupData = groupData.rename(columns = {"size":"MAX_KNOB"})

# 병합할 키 설정
joinKey = ["국가", "분야"]

# 데이터 병합
mergedData = pd.merge(left=train, right=groupData, on=joinKey)

# 결과 확인
mergedData.sort_values(by="국가", ascending=False)

,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률,MAX_KNOB
1893,2017,CT010,푸드테크,7.600902,4412,0,1,4.0,103.0,8167.0,1.0,5250.0,0.6,27
2109,2008,CT010,이커머스,7.600902,327,0,1,7.0,3352.0,11569.0,4.0,5250.0,0.8,19
1578,2009,CT010,에너지,5.164786,4920,1,1,97.0,419.0,4109.0,3.0,5250.0,0.7,47
2085,2011,CT010,이커머스,6.214608,4172,1,1,27.0,4919.0,11300.0,2.0,6000.0,0.5,19
2089,2001,CT010,물류,7.600902,1786,0,0,51.0,6096.0,12355.0,2.0,5250.0,0.7,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,2012,CT001,푸드테크,5.164786,881,1,1,78.0,69.0,4711.0,4.0,3000.0,0.7,32
1599,2013,CT001,게임,7.600902,2996,0,1,47.0,1654.0,10355.0,2.0,5250.0,0.6,22
1938,2022,CT001,게임,7.600902,2023,1,1,7.0,4082.0,11136.0,1.0,4000.0,0.1,22
1936,2014,CT001,기술,6.214608,422,0,1,71.0,962.0,3673.0,1.0,2000.0,0.7,45


In [ ]:
# mergedData.MAX_KNOB.value_counts().sort_values()

In [ ]:
# 75번째 백분위수 값을 기준으로 설정

threshold = mergedData['MAX_KNOB'].quantile(0.5)
threshold

np.float64(27.0)

In [ ]:
mergedData.groupby(groupKey)

In [ ]:
stdMaxKnob = threshold  # 제외 기준 기업 수

# 머신러닝 제외 아이템 선별
# dropData = mergedData.loc[mergedData['MAX_KNOB'] < stdMaxKnob]
# dropData
# 제외 대상 아이템 선택: '국가'와 '분야' 컬럼을 기준으로 고유 값 추출
# targetItem = ["국가", "분야"]
# exceptItemDf = dropData.loc[:, targetItem].drop_duplicates()

# 제외 아이템을 CSV로 저장
# exceptItemDf.to_csv("exceptItemList.csv", index=False)

# 머신러닝 대상 아이템 선별
refinedData = mergedData.loc[mergedData['MAX_KNOB'] >= stdMaxKnob]
refinedData
# 결과 확인
# print(f"Excluded items (to be saved in exceptItemList.csv):\n{exceptItemDf.head()}")
# print(f"Refined data (for machine learning):\n{refinedData.head()}")


,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률,MAX_KNOB
0,2023,CT006,핀테크,1.609438,4167,1,0,80.0,4069.0,279.0,1.0,3000.0,0.8,27
1,2018,CT007,기술,3.912023,3132,1,1,54.0,6453.0,12141.0,4.0,4000.0,0.5,37
2,2020,CT002,에듀테크,1.609438,1969,0,1,94.0,829.0,9810.0,1.0,2000.0,0.1,32
3,2012,CT008,기술,6.214608,3801,0,1,69.0,6505.0,12722.0,3.0,3000.0,0.6,34
5,2015,CT001,이커머스,3.912023,2846,1,1,86.0,3504.0,1067.0,3.0,5250.0,0.7,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2518,2012,CT004,에너지,7.600902,2027,1,0,46.0,5648.0,6247.0,1.0,3000.0,0.4,28
2519,2019,CT009,핀테크,7.600902,2098,0,0,36.0,336.0,9455.0,0.0,5250.0,0.4,41
2522,2013,CT001,에듀테크,5.164786,3112,1,0,62.0,5447.0,6773.0,2.0,4000.0,0.4,30
2525,2020,CT003,에너지,5.164786,555,0,1,37.0,796.0,2969.0,3.0,6000.0,0.4,33


In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # 국가 분포 시각화
# plt.figure(figsize=(10, 5))
# sns.countplot(data=train, x="국가", order=train["국가"].value_counts().index)
# plt.title("국가별 데이터 분포")
# plt.xticks(rotation=45)
# plt.show()

# # 분야 분포 시각화
# plt.figure(figsize=(10, 5))
# sns.countplot(data=train, x="분야", order=train["분야"].value_counts().index)
# plt.title("분야별 데이터 분포")
# plt.xticks(rotation=45)
# plt.show()


In [ ]:
# num_cols = ["연매출(억원)", "총 투자금(억원)", "성공확률", "직원 수", "기업가치(백억원)"]

# for col in num_cols:
#     plt.figure(figsize=(6, 4))
#     sns.histplot(data=train, x=col, kde=True)
#     plt.title(f"{col} 분포")
#     plt.show()


In [ ]:
# groupData = train.groupby(["국가", "분야"]).size().reset_index(name="기업수")

# plt.figure(figsize=(12, 6))
# sns.boxplot(x="국가", y="기업수", data=groupData)
# plt.title("국가별 기업 수 분포")
# plt.xticks(rotation=45)
# plt.show()


In [ ]:
groupKey = ["국가", "분야"]

In [ ]:
groupData = train.groupby( groupKey )

In [ ]:
groupData.head()

,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
1,2023,CT006,핀테크,1.609438,4167,1,0,80.0,4069.0,279.0,1.0,3000.0,0.8
2,2018,CT007,기술,3.912023,3132,1,1,54.0,6453.0,12141.0,4.0,4000.0,0.5
4,2020,CT002,에듀테크,1.609438,1969,0,1,94.0,829.0,9810.0,1.0,2000.0,0.1
5,2012,CT008,기술,6.214608,3801,0,1,69.0,6505.0,12722.0,3.0,3000.0,0.6
7,2020,CT002,게임,3.912023,2617,0,0,80.0,5579.0,5399.0,4.0,4000.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,2011,CT009,헬스케어,6.214608,2495,0,0,95.0,5041.0,6800.0,3.0,6000.0,0.7
1937,2015,CT005,이커머스,3.912023,3923,0,0,41.0,694.0,10463.0,1.0,6000.0,0.9
2098,2022,CT010,물류,1.609438,3436,0,0,82.0,6125.0,11063.0,2.0,3000.0,0.1
2170,2015,CT006,기술,7.600902,4617,0,0,25.0,2948.0,2114.0,2.0,6000.0,0.9


In [ ]:

# eachGroup = groupData.reset_index(drop=True)
corr = groupData.corr(numeric_only = True)
# round(abs(corr['성공확률'].drop('성공확률')),4).sort_values(ascending=False)


# # corr
# target_corr = round(abs(corr['성공확률'].drop('성공확률')),4).sort_values(ascending=False)
# target_corr

In [ ]:
# 조건에 맞는 상관계수 필터링
filtered_corr = corr[(corr['성공확률'] > 0.2) & (corr['성공확률'] != 1)]["성공확률"]

# 결과 출력
filtered_corr


국가     분야                  
CT001  AI    투자단계              0.383763
             직원 수              0.259665
             총 투자금(억원)         0.306336
             SNS 팔로워 수(백만명)    0.389331
       게임    투자단계              0.250593
                                 ...   
CT010  푸드테크  연매출(억원)           0.253101
       핀테크   투자단계              0.454768
             직원 수              0.282560
             상장여부              0.600481
       헬스케어  연매출(억원)           0.290843
Name: 성공확률, Length: 171, dtype: float64

In [ ]:
filtered_corr.to_csv("filtered_corr.csv")

In [ ]:

# modelDict = {
#     'Decision Tree': DecisionTreeRegressor(random_state=10),
#     'Random Forest': RandomForestRegressor(n_estimators=100, random_state=10),
#     'Linear Regression': LinearRegression(),
#     'TabNet Regression': TabNetRegressor(verbose=0, seed = 10, optimizer = torch.optim.AdamW)
# }


In [ ]:
modelDict = {}

In [ ]:
def mlFunc( eachGroup ):
    # eachGroup = refinedData.loc[ (refinedData.REGIONID == "A01") &
    #                 (refinedData.ITEM == "ITEM013") ]
    ### 2. 특성선정 및 데이터분리
    eachGroup = eachGroup.reset_index(drop=True)
    corrDf = eachGroup.corr(numeric_only = True)
    countryId = eachGroup.loc[0, ["국가"]].values[0]
    categoryId = eachGroup.loc[0, ["분야"]].values[0]
    stdCorr = 0.3
    label=["성공확률"]
    features = list( corrDf.loc[ (abs( corrDf.성공확률 )  >= stdCorr) &
                (abs( corrDf.성공확률 ) != 1) ].index )
    if not features:
        return None  # 예외 처리
    print(f"{countryId} {categoryId}  선정된 특성 {features}")
    # features
    from sklearn.model_selection import train_test_split
    trainData, testData  = train_test_split( eachGroup, test_size=0.2, random_state=10 )
    trainDataFeatures = trainData.loc[ :, features ]
    trainDataLabel = trainData.loc[ :, label ]
    testDataFeatures = testData.loc[ :, features ]
    testDataLabel = testData.loc[ :, label ]
    testDataAll = testData.loc[ : ]
    # print(trainDataFeatures.shape)
    # print(trainDataLabel.shape)
    # print(testDataFeatures.shape)
    # print(testDataLabel.shape)
    ### 3. 모델 적용
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.linear_model import LinearRegression
    dtModel = DecisionTreeRegressor(random_state=10)
    rfModel = RandomForestRegressor(n_estimators=100, random_state=10)
    lrModel = LinearRegression()

    fittedDtModel = dtModel.fit(trainDataFeatures, trainDataLabel)
    fittedRfModel = rfModel.fit(trainDataFeatures, trainDataLabel)
    fittedLrModel = lrModel.fit(trainDataFeatures, trainDataLabel)
    ### 4. 예측
    predictDtValue = fittedDtModel.predict( testDataFeatures )
    predictRfValue = fittedRfModel.predict( testDataFeatures )
    predictLrValue = fittedLrModel.predict( testDataFeatures )
    ### 5. 데이터 정리
    testDataAll["DT_PREDICT"] = predictDtValue
    testDataAll["RF_PREDICT"] = predictRfValue
    testDataAll["LR_PREDICT"] = predictLrValue
    testDataAll["features"] = str( features )

    ### 모델 저장
    modelDict[ countryId,categoryId ] = fittedLrModel

    return testDataAll

In [ ]:
finalResult = mergedData.groupby( groupKey ).apply( mlFunc )

CT001 AI  선정된 특성 ['투자단계', '직원 수', '총 투자금(억원)', 'SNS 팔로워 수(백만명)']
CT001 게임  선정된 특성 ['투자단계', '인수여부', '고객수(백만명)', '총 투자금(억원)']
CT001 기술  선정된 특성 ['기업가치(백억원)']
CT001 물류  선정된 특성 ['고객수(백만명)', '기업가치(백억원)']
CT001 에너지  선정된 특성 ['직원 수']
CT001 에듀테크  선정된 특성 ['인수여부', '상장여부']
CT001 이커머스  선정된 특성 ['투자단계', '직원 수', '인수여부', '총 투자금(억원)', 'SNS 팔로워 수(백만명)']
CT001 푸드테크  선정된 특성 ['직원 수', '인수여부', '총 투자금(억원)', '연매출(억원)']
CT001 핀테크  선정된 특성 ['직원 수', '연매출(억원)', '기업가치(백억원)']
CT001 헬스케어  선정된 특성 ['투자단계', '직원 수', '인수여부', '상장여부']
CT002 AI  선정된 특성 ['설립연도', '투자단계', '인수여부', '연매출(억원)']
CT002 게임  선정된 특성 ['직원 수', '고객수(백만명)', '총 투자금(억원)']
CT002 기술  선정된 특성 ['투자단계', '직원 수', '인수여부', '고객수(백만명)', 'SNS 팔로워 수(백만명)', '기업가치(백억원)']
CT002 물류  선정된 특성 ['투자단계', '인수여부', '상장여부', '고객수(백만명)', '기업가치(백억원)']
CT002 에너지  선정된 특성 ['상장여부', 'SNS 팔로워 수(백만명)']
CT002 에듀테크  선정된 특성 ['총 투자금(억원)']
CT002 이커머스  선정된 특성 ['투자단계', '인수여부', '연매출(억원)']
CT002 푸드테크  선정된 특성 ['투자단계', '상장여부', '고객수(백만명)']
CT002 핀테크  선정된 특성 ['인수여부', '연매출(억원)', '기업가치(백억원)']
CT002 헬스케어  선정된 특성 ['

In [ ]:
finalMlResult = finalResult.reset_index(drop=True)
finalMlResult.head()

,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률,MAX_KNOB,DT_PREDICT,RF_PREDICT,LR_PREDICT,features
0,2012,CT001,AI,1.609438,2283,1,0,83.0,5613.0,1909.0,3.0,3000.0,0.5,21,0.8,0.685,0.649608,"['투자단계', '직원 수', '총 투자금(억원)', 'SNS 팔로워 수(백만명)']"
1,2007,CT001,AI,5.164786,4737,0,0,84.0,2713.0,10516.0,1.0,2000.0,0.6,21,0.5,0.476,0.567003,"['투자단계', '직원 수', '총 투자금(억원)', 'SNS 팔로워 수(백만명)']"
2,2001,CT001,AI,5.164786,4155,1,0,52.0,125.0,10157.0,2.0,2000.0,0.5,21,0.7,0.644,0.536816,"['투자단계', '직원 수', '총 투자금(억원)', 'SNS 팔로워 수(백만명)']"
3,2016,CT001,AI,3.912023,694,1,1,16.0,3514.0,7838.0,0.0,6000.0,0.4,21,0.8,0.593,0.583484,"['투자단계', '직원 수', '총 투자금(억원)', 'SNS 팔로워 수(백만명)']"
4,2008,CT001,AI,6.214608,4490,1,1,17.0,5232.0,7955.0,1.0,6000.0,0.7,21,0.8,0.627,0.668911,"['투자단계', '직원 수', '총 투자금(억원)', 'SNS 팔로워 수(백만명)']"


In [ ]:
finalMlResult.loc[(finalMlResult.국가 == "CT010") & (finalMlResult.분야 == "AI")]

,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률,MAX_KNOB,DT_PREDICT,RF_PREDICT,LR_PREDICT,features


In [ ]:
# 예: 입력값 (features는 이 모델이 학습할 때 사용된 컬럼들과 같아야 함)
input_data = pd.DataFrame([[ 1.609438, 2283	, 5613.0, 3.0]], columns=["투자단계", "직원 수", "총 투자금(억원)", "SNS 팔로워 수(백만명)"])
# loaded_model = modelDict[("CT001", "AI")]


# 예측
try:
    loaded_model = modelDict[("CT010", "AI")]
    prediction = loaded_model.predict(input_data)
    print("예측된 성공확률:", prediction[0])
except:
    print("there's not sufficient data for this prediction")


there's not sufficient data for this prediction


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
dt_mae = mean_absolute_error( y_true=finalMlResult.성공확률, y_pred=finalMlResult.DT_PREDICT )
rf_mae = mean_absolute_error( y_true=finalMlResult.성공확률, y_pred=finalMlResult.RF_PREDICT )
lr_mae = mean_absolute_error( y_true=finalMlResult.성공확률, y_pred=finalMlResult.LR_PREDICT )

In [ ]:
maeResult = pd.DataFrame( [[dt_mae, rf_mae, lr_mae]] , columns=["dt","rf","lr"])
maeResult

,dt,rf,lr
0,0.238193,0.199448,0.199202


In [ ]:
modelDict

{('CT001', 'AI'): LinearRegression(),
 ('CT001', '게임'): LinearRegression(),
 ('CT001', '기술'): LinearRegression(),
 ('CT001', '물류'): LinearRegression(),
 ('CT001', '에너지'): LinearRegression(),
 ('CT001', '에듀테크'): LinearRegression(),
 ('CT001', '이커머스'): LinearRegression(),
 ('CT001', '푸드테크'): LinearRegression(),
 ('CT001', '핀테크'): LinearRegression(),
 ('CT001', '헬스케어'): LinearRegression(),
 ('CT002', 'AI'): LinearRegression(),
 ('CT002', '게임'): LinearRegression(),
 ('CT002', '기술'): LinearRegression(),
 ('CT002', '물류'): LinearRegression(),
 ('CT002', '에너지'): LinearRegression(),
 ('CT002', '에듀테크'): LinearRegression(),
 ('CT002', '이커머스'): LinearRegression(),
 ('CT002', '푸드테크'): LinearRegression(),
 ('CT002', '핀테크'): LinearRegression(),
 ('CT002', '헬스케어'): LinearRegression(),
 ('CT003', 'AI'): LinearRegression(),
 ('CT003', '게임'): LinearRegression(),
 ('CT003', '기술'): LinearRegression(),
 ('CT003', '물류'): LinearRegression(),
 ('CT003', '에너지'): LinearRegression(),
 ('CT003', '에듀테크'): LinearReg

In [ ]:
corr = train.corr(numeric_only=True)#, method = 'pearson')
# corr
target_corr = round(abs(corr['성공확률'].drop('성공확률')),4).sort_values(ascending=False)
target_corr

,성공확률
상장여부,0.0543
설립연도,0.0354
기업가치(백억원),0.0344
총 투자금(억원),0.0318
SNS 팔로워 수(백만명),0.0308
고객수(백만명),0.0202
연매출(억원),0.0188
직원 수,0.0182
인수여부,0.0166
투자단계,0.0012


# Train

In [ ]:
label = ["성공확률"]
features = train.columns.difference(label)

Index(['SNS 팔로워 수(백만명)', '고객수(백만명)', '국가', '기업가치(백억원)', '분야', '상장여부', '설립연도',
       '연매출(억원)', '인수여부', '직원 수', '총 투자금(억원)', '투자단계'],
      dtype='object')

In [ ]:
trainData, testData = train_test_split(train, test_size=0.2, random_state=10)

In [ ]:
# trainDataFeatures = trainData.loc[ :, features ]
# trainDataLabels = trainData.loc[ :, label ]
trainDataFeatures = trainData.loc[ :, features ]
trainDataLabels = trainData.loc[ :, label ]
testDataFeatures = testData.loc[ :, features ]
testDataLabels = testData.loc[ :, label ]
testDataAll = testData.loc[ :, : ]

print(trainDataFeatures.shape)
print(trainDataLabels.shape)
print(testDataFeatures.shape)
print(testDataLabels.shape)
print(testDataAll.shape)

(2022, 12)
(2022, 1)
(506, 12)
(506, 1)
(506, 13)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
models = {
    'Decision Tree': DecisionTreeRegressor(random_state=10),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=10),
    'Linear Regression': LinearRegression()
}

In [ ]:
import joblib

for name, model in models.items():
    print(f"\n===== {name} =====")
    val_name = name.replace(" ", "_")
    final_model = model.fit(X = trainDataFeatures, y = trainDataLabels)
    joblib.dump(final_model, f"/content/mlcore_{model}.dump")
    y_pred = model.predict(testDataFeatures)
    testDataAll[val_name] = y_pred


===== Decision Tree =====

===== Random Forest =====

===== Linear Regression =====


In [ ]:
testDataAll

,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률,Decision_Tree,Random_Forest,Linear_Regression
1513,2022,2,8,4,1609,1,0,19.0,3044.0,4337.0,2.0,2000.0,0.4,0.6,0.491,0.482111
568,2010,9,1,1,3990,0,0,22.0,3940.0,11866.0,4.0,4000.0,0.8,0.9,0.587,0.563629
3996,2017,9,0,1,4207,1,1,62.0,1117.0,3958.0,2.0,3000.0,0.7,0.7,0.579,0.565547
4165,2016,4,0,2,1796,1,0,28.0,3583.0,7636.0,4.0,2000.0,0.8,0.3,0.522,0.522060
2904,2023,6,4,0,3451,1,1,86.0,114.0,3924.0,4.0,3000.0,0.8,0.8,0.598,0.561715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,2003,1,9,1,3921,1,0,74.0,1407.0,2969.0,4.0,2000.0,0.4,0.5,0.422,0.522393
229,2005,1,8,1,4971,1,0,68.0,969.0,8196.0,3.0,5250.0,0.4,0.4,0.625,0.529838
1424,2006,2,2,0,3393,1,0,87.0,1418.0,9574.0,3.0,3000.0,0.7,0.7,0.578,0.528663
3865,2012,2,3,0,3200,1,0,48.0,5713.0,11953.0,1.0,5250.0,0.6,0.8,0.534,0.505539


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
dt_mae = mean_absolute_error( y_true=testDataAll.성공확률, y_pred=testDataAll.Decision_Tree )
rf_mae = mean_absolute_error( y_true=testDataAll.성공확률, y_pred=testDataAll.Random_Forest )
lr_mae = mean_absolute_error( y_true=testDataAll.성공확률, y_pred=testDataAll.Linear_Regression )

In [ ]:
maeResult = pd.DataFrame( [[dt_mae, rf_mae, lr_mae]] , columns=["dt","rf","lr"])
maeResult

,dt,rf,lr
0,0.237945,0.180053,0.189324
